### Parallel response from api

In [21]:
places_api_key = "your google places api key"
geocoding_api = "your google geocoding api key"

In [3]:
import requests
from urllib.parse import urlencode
from math import radians, cos, sin, asin, sqrt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [11]:
def distance(lat1, lon1, lat2, lon2):

    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
    
    # calculate the result
    return(c * r)

In [12]:
####### distance measurement

distance(23.81672, 90.39605999999999, 23.81672, 90.3644)

lat1 radian 0.4156801810255839


3.2206380063167432

In [24]:
def extract_lat_lng(address_or_postalcode, data_type = 'json'):
    endpoint = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
    params = {"address": address_or_postalcode, "key": geocoding_api}
    url_params = urlencode(params)
    url = f"{endpoint}?{url_params}"
    r = requests.get(url)
    if r.status_code not in range(200, 299): 
        return {}
    latlng = {}
    try:
        latlng = r.json()['results'][0]['geometry']['location']
    except:
        pass
    return latlng.get("lat"), latlng.get("lng")

In [25]:
def search_places(place_type, searching_radius, lat, lng):
    base_endpoint_places = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        "key": places_api_key,
        "input": place_type,
        "inputtype": "textquery",
        "fields": "place_id,formatted_address,name,geometry,permanently_closed"
    }
    locationbias = f"point:{lat},{lng}"
    use_cirular = True
    if use_cirular:
        radius = searching_radius
        locationbias = f"circle:{radius}@{lat},{lng}"

    params['locationbias'] = locationbias

    params_encoded = urlencode(params)
    places_endpoint = f"{base_endpoint_places}?{params_encoded}"
    # print(places_endpoint)

    r = requests.get(places_endpoint)
    # print(r.status_code)
    return r.json()

In [26]:
def rectangle_four_coordinates(midpoint_lat, midpoint_lng):
    rightupperpoint_lat, rightupperpoint_lng = midpoint_lat+0.02500, midpoint_lng+0.01583
    upperleftpoint_lat, upperleftpoint_lng = midpoint_lat+0.02500, midpoint_lng-0.01583
    leftdownpoint_lat, leftdownpoint_lng = midpoint_lat-0.01357, midpoint_lng-0.01583
    downrightpoint_lat, downrightpoint_lng = midpoint_lat-0.01357, midpoint_lng+0.01583

    return rightupperpoint_lat, rightupperpoint_lng, upperleftpoint_lat, upperleftpoint_lng,  leftdownpoint_lat, leftdownpoint_lng, downrightpoint_lat, downrightpoint_lng

In [27]:
midpoint_lat, midpoint_lng = 23.79172, 90.38023

rightupperpoint_lat, rightupperpoint_lng, upperleftpoint_lat, upperleftpoint_lng,  leftdownpoint_lat, leftdownpoint_lng, downrightpoint_lat, downrightpoint_lng = rectangle_four_coordinates(midpoint_lat, midpoint_lng)

print(rightupperpoint_lat, rightupperpoint_lng, upperleftpoint_lat, upperleftpoint_lng,  leftdownpoint_lat, leftdownpoint_lng, downrightpoint_lat, downrightpoint_lng)

23.81672 90.39605999999999 23.81672 90.3644 23.77815 90.3644 23.77815 90.39605999999999


In [50]:
def geofencing(location):
    polygon = Polygon([
        (rightupperpoint_lat, rightupperpoint_lng),
        (upperleftpoint_lat, upperleftpoint_lng),
        (leftdownpoint_lat, leftdownpoint_lng),
        (downrightpoint_lat, downrightpoint_lng),
        (rightupperpoint_lat, rightupperpoint_lng)
    ])
    # print("############", i)
    point = Point(location)

    if polygon.contains(point):
        return point.x, point.y
    

In [53]:
from logging import exception


place_type = ['mosque']
searching_radius = 3400 #meters

# place = set()
locations = dict()

for i in range(0,searching_radius,200):
    for type in place_type:
        place_name = search_places(type, i, midpoint_lat, midpoint_lng)
        # print("Place Name: ", place_name)
        try:
            place = place_name['candidates'][0]['name']
            place = place+', Bangladesh'
            # print(place)
            location = extract_lat_lng(place)
            # print("location:::::",location)
            # print(type(location))
            # print(f'locations[0]: {locations[0]},location[1]: {location[1]}')
            # for i,j in location:
            #     print(f"{i},{j}")
            box_inner_locations = geofencing(location)
            

            print("box_inner_locations", box_inner_locations)
            loc = locations.get(box_inner_locations, -1)
            print("Print loc:",box_inner_locations)
            if loc == -1 and box_inner_locations:
                locations[box_inner_locations] = box_inner_locations
            else:
                pass
        except Exception as e:
            print(e)
            # pass
            
        # schools.add(school_name+", Bangladesh")

        
        # print(locations)


box_inner_locations None
Print loc: None
box_inner_locations (23.7928489, 90.37749409999999)
Print loc: (23.7928489, 90.37749409999999)
box_inner_locations (23.7928489, 90.37749409999999)
Print loc: (23.7928489, 90.37749409999999)
box_inner_locations (23.7914582, 90.37499489999999)
Print loc: (23.7914582, 90.37499489999999)
box_inner_locations (23.7877396, 90.38651759999999)
Print loc: (23.7877396, 90.38651759999999)
box_inner_locations (23.7877396, 90.38651759999999)
Print loc: (23.7877396, 90.38651759999999)
box_inner_locations (23.7877396, 90.38651759999999)
Print loc: (23.7877396, 90.38651759999999)
box_inner_locations (23.7877396, 90.38651759999999)
Print loc: (23.7877396, 90.38651759999999)
box_inner_locations (23.7877396, 90.38651759999999)
Print loc: (23.7877396, 90.38651759999999)
box_inner_locations (23.7877396, 90.38651759999999)
Print loc: (23.7877396, 90.38651759999999)
box_inner_locations (23.7877396, 90.38651759999999)
Print loc: (23.7877396, 90.38651759999999)
box_inner

In [54]:
print(locations)

{(23.7928489, 90.37749409999999): (23.7928489, 90.37749409999999), (23.7914582, 90.37499489999999): (23.7914582, 90.37499489999999), (23.7877396, 90.38651759999999): (23.7877396, 90.38651759999999)}
